Figure in Page 4
    - Lambda values --> what exactly is being varied?
        - lambda used to vary beta.

In [76]:
import numpy as np
import matplotlib.pyplot as plt

M = np.array([[19.2, 4.8, 3.0, 7.1, 3.7, 3.1, 2.3, 1.4, 1.4],
[4.8, 42.4, 6.4, 5.4, 7.5, 5.0, 1.8, 1.7, 1.7],
[3.0, 6.4, 20.7, 9.2, 7.1, 6.3, 2.0, 0.9, 0.9],
[7.1, 5.4, 9.2, 16.9, 10.1, 6.8, 3.4, 1.5, 1.5],
[3.7, 7.5, 7.1, 10.1, 13.1, 7.4, 2.6, 2.1, 2.1],
[3.1, 5.0, 6.3, 6.8, 7.4, 10.4, 3.5, 1.8, 1.8],
[2.3, 1.8, 2.0, 3.4, 2.6, 3.5, 7.5, 3.2, 3.2],
[1.4, 1.7, 0.9, 1.5, 2.1, 1.8, 3.2, 7.2, 7.2],
[1.4, 1.7, 0.9, 1.5, 2.1, 1.8, 3.2, 7.2, 7.2]]
)

S = np.array([900 for i in range(9)])
I = np.array([100 for i in range(9)])
R = np.array([0 for i in range(9)])

gamma = 1/14


def exp(lambda_):
    return float('inf') if lambda_ == 0 else np.random.exponential(1 / lambda_)


# def dS(beta, Si, N, M, I, i): # M is an i by j matrix, I is a 9 length vector
#   return -beta * Si / N * np.sum( M[i] * I[j] )


# def dI(beta, Si, N, M, I, i, gamma): # M is an i by j matrix, I is a 9 length vector
#   return beta * Si / N * np.sum( M[i] * I[j]) - gamma * I[i]


# def dR(gamma, Ii): # I is a 9 length vector
#   return gamma * Ii


def dSI(beta, Si, N, Mi, I):
  return beta * Si / N * np.dot(Mi, I)

def dRI(gamma, Ii):
  return gamma * Ii


In [77]:
#where i = the ith age range
h = [.1, .3, 1.2, 3.2, 4.9, 10.2, 16.6, 24.3, 27.3] #hospital
c = [5.0, 5.0, 5.0, 5.0, 6.3, 12.2, 27.4, 43.2, 70.9] #icu
m = [0.002, .006, .03, .08, .15, .6, 2.2, 5.1, 9.3] #morir

In [78]:
def dH(gamma, Ii, Hi):
  return gamma * Hi * Ii

def dC(gamma, Ii, Hi, ci):
  return gamma * Hi * Ii * ci

def dM(gamma, mi, Ii):
  return gamma * mi * Ii

In [80]:
def modelloop(L, N, i):
  beta = 5.7 * gamma / L #L - lambda
  # I = 100 #originally i
  # S = N - i #healthy
  # R = 0 #recovered
  while np.sum(I) > 0:
    #for each age range i, we calculate step from S to I, and then again from I to R.
    betaoutcomes, gammaoutcomes = np.array([dSI(beta, S[i], N, M[i], I[i]) for i in range(9)]), np.array([dRI(gamma, I[i]) for i in range(9)]) # calculate equation values
    #get the minimum value over all, make updates to S, I, and R based on min
    # print(betaoutcomes, gammaoutcomes)
    #print(np.concatenate(betaoutcomes, gammaoutcomes))
    #min_val = get_min_ind(betaoutcomes, gammaoutcomes)
    min_val = np.argmin(np.concatenate(betaoutcomes, gammaoutcomes))
    if min_val < 9: #then move person from S to I
      idx = min_val
      S[idx] -= 1
      I[idx] += 1
    else: #the move person from I to S
      idx = min_val - 9
      I[idx] -= 1
      R[idx] += 1

In [81]:
#for i in range(9):
modelloop(.01, 900, 1)
plt.plot(I)


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()